In [1]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import date, timedelta
import calendar

pd.options.display.max_rows = 30

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

format_dict = {'Q4':'{:.4f}','Q3':'{:.4f}','Q2':'{:.4f}','Q1':'{:.4f}','qtr':'{:.4f}',
               'div_amt':'{:,.2f}','net_div':'{:,.2f}','cost_amt':'{:,.2f}',
               'price':'{:.2f}','unit_cost':'{:.2f}','pct':'{:.2f}%','shares':'{:,}','yield':'{:.2f}%',
              }
data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
today = date.today()
today

datetime.date(2022, 1, 5)

### Start of New buy record

In [5]:
name = "IMH"
new_cost = 11
new_qty = 9000

In [9]:
sql = """
SELECT * 
FROM buy 
WHERE active = 1 AND name = '%s'
ORDER BY name"""
sql = sql % name
df = pd.read_sql(sql, const)
df.set_index('name', inplace=True)
df['volume'] = df.volbuy.astype(int)
cols = 'date volume price active dividend period grade'.split()
df[cols]

,date,volume,price,active,dividend,period,grade
name,,,,,,,
IMH,2021-12-08,9000,11.0,1,0.0,3,A3


In [7]:
sql = """
INSERT INTO buy 
VALUES('IMH','2021-12-08',9000,11,0,0,1,0,'3','A3')
"""
print(sql)


INSERT INTO buy 
VALUES('IMH','2021-12-08',9000,11,0,0,1,0,'3','A3')



In [8]:
rp = const.execute(sql)
rp.rowcount

1

### End of new buy

### Start of buy update

In [3]:
def update_buy(shares,price,name,period):
    sql = "UPDATE buy SET volbuy = %s, price = %s WHERE name = '%s' AND period = '%s'"
    sql = sql % (shares, price, name, period)
    rp = const.execute(sql)
    return "Records updated = " + str(rp.rowcount)

In [7]:
name = 'RJH'
sql = """
SELECT * 
FROM buy 
WHERE active = 1 AND name = '%s'
ORDER BY name"""
sql = sql % name
df = pd.read_sql(sql, const)
df.set_index('name', inplace=True)
df['volume'] = df.volbuy.astype(int)
cols = 'date volume price active dividend period grade'.split()
df[cols]

,date,volume,price,active,dividend,period,grade
name,,,,,,,
RJH,2021-11-09,4500,33.0,1,1.7,3,A3


In [5]:
shares = 4500
price =  33
period = '3'
(name, shares, price, period)

('RJH', 4500, 33, '3')

In [6]:
update_buy(shares,price,name,period)

'Records updated = 1'

### Change grade from quarterly performance

In [2]:
def update_buy_grade(grade,period,name):
    sql = "UPDATE buy SET grade = '%s', period = %s WHERE name = '%s' AND active=1"
    sql = sql % (grade, period, name)
    rp = const.execute(sql)
    return "Records updated = " + str(rp.rowcount)

In [17]:
name = 'RCL'
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
df_tmp = pd.read_sql(sql, const)
df_tmp['volume'] = df_tmp.volbuy.astype(int)
cols = 'date volume price active dividend period grade'.split()
df_tmp[cols]

,date,volume,price,active,dividend,period,grade
0,2021-11-08,4500,43.0,1,3.5,3,A4


In [16]:
grade = 'A4'
period = '3'

update_buy_grade(grade,period,name)

'Records updated = 1'

### End of Buy update

### When sold out

In [3]:
name = 'LH'
sql = '''
DELETE FROM buy 
WHERE name = "%s"
AND period = "1"'''
sql = sql % name
print(sql)


DELETE FROM buy 
WHERE name = "LH"
AND period = "1"


In [4]:
rp = const.execute(sql)
rp.rowcount

1

### End of sold out

In [7]:
sql = """
SELECT * 
FROM buy 
WHERE active = 1
ORDER BY name"""
df = pd.read_sql(sql, const)
df.set_index('name', inplace=True)
df['volume'] = df.volbuy.astype(int)
cols = 'date volume price active dividend period grade'.split()
df[cols].loc['HREIT']

date        2020-08-14
volume           20000
price             8.03
active               1
dividend        0.6909
period               4
grade               A1
Name: HREIT, dtype: object

### New dividend stock

In [8]:
pd.set_option('display.float_format','{:,.6f}'.format)

In [2]:
name = 'WHART'
sql = """
SELECT * 
FROM DIVIDEND
WHERE name = '%s'
"""
sql = sql % name
df = pd.read_sql(sql, const)
df

,NAME,Q4,Q3,Q2,Q1,DIVIDEND,PRICE,PERCENT,SHARES,XDATE,PAIDDATE,KIND,ACTUAL
0,WHART,0.1915,0.2553,0.1915,0.1915,0.8298,0.0,0.0,10000.0,2022-03-04,2021-12-08,,0


In [19]:
sql = """
INSERT INTO dividend
VALUES('KBANK',2.50,0.00,0.50,0.0,3.00,0,0,000,'2022-04-20','2022-05-07','',0)
"""
rp = const.execute(sql)
rp.rowcount

1

### Start of Update dividend

In [8]:
def update_shares(shares,actual,name):
    sql = "UPDATE dividend SET shares = %s, actual = %s WHERE name = '%s'"
    sql = sql % (shares, actual, name)
    rp = const.execute(sql)
    return "Records updated = " + str(rp.rowcount)

In [11]:
name = 'RJH'
sql = """
SELECT * 
FROM DIVIDEND
WHERE name = '%s'
"""
sql = sql % name
df = pd.read_sql(sql, const)
df['SHARES'] = df['SHARES'].astype(int)
df.style.format(format_dict)

,NAME,Q4,Q3,Q2,Q1,DIVIDEND,PRICE,PERCENT,SHARES,XDATE,PAIDDATE,KIND,ACTUAL
0,RJH,0.2000,1.1000,0.4000,0.0000,1.7000,0.00,0.00%,"4,500",2021-11-22,2021-12-07,,0


In [10]:
shares = 4500
actual = 0
update_shares(shares,actual,name)

'Records updated = 1'

### Update total dividend from dividend file to buy file

In [17]:
sql = '''
UPDATE buy B
SET dividend =
(SELECT DIVIDEND FROM dividend D
WHERE B.name = D.name)'''
rp = const.execute(sql)
rp.rowcount

35

### Delete dividend record

In [5]:
sql = '''
DELETE FROM dividend
WHERE name IN ("LH")
'''
rp = const.execute(sql)
rp.rowcount

1

### End of Delete dividend record

### Calculate dividend amt by quarter

In [2]:
sql = """
SELECT D.NAME AS name, xdate as xd_date, paiddate AS pay_date, Q4 AS qtr,
SHARES AS shares, B.volbuy, Q4*SHARES AS div_amt
FROM dividend D
LEFT JOIN buy B
ON D.name = B.name
WHERE Q4 > 0 AND shares > 0
ORDER BY D.name"""
df_tmp = pd.read_sql(sql, const)
df_tmp[df_tmp['volbuy'].isna()]

,name,xd_date,pay_date,qtr,shares,volbuy,div_amt


In [14]:
names = df_tmp[df_tmp['volbuy'].isna()]['name']
in_p = ', '.join(map(lambda name: "'%s'" % name,names))
in_p

''

In [15]:
sqlUpd  = '''
UPDATE dividend
SET shares = 0
WHERE name IN (%s)'''
sqlUpd = sqlUpd % in_p
#rp = const.execute(sqlUpd)
#rp.rowcount

In [2]:
sql = """
SELECT NAME AS name, Q4 AS qtr, SHARES AS shares, Q4*SHARES AS div_amt,
(Q4*SHARES*.9) AS net_div, xdate as xd_date, paiddate AS pay_date
FROM dividend
WHERE Q4 > 0 AND shares > 0
ORDER BY name"""
df_div = pd.read_sql(sql, const)
df_div.set_index('name', inplace=True)
df_div['shares'] = df_div['shares'].astype(int)
df_div.sort_values(['div_amt'],ascending=[False]).style.format(format_dict)

,qtr,shares,div_amt,net_div,xd_date,pay_date
name,,,,,,
MCS,0.6000,"60,000","36,000.00","32,400.00",2021-11-25,2021-12-09
STA,1.7500,"20,000","35,000.00","31,500.00",2021-11-19,2021-12-08
NOBLE,0.5000,"63,000","31,500.00","28,350.00",2021-08-24,2021-09-07
JASIF,0.2500,"100,000","25,000.00","22,500.00",2021-11-19,2021-12-08
DIF,0.2610,"60,000","15,660.00","14,094.00",2021-11-11,2021-12-03
TMT,0.5000,"27,000","13,500.00","12,150.00",2021-08-27,2021-09-10
TISCO,6.3000,"2,000","12,600.00","11,340.00",2021-04-28,2021-05-17
RATCH,1.2500,"9,000","11,250.00","10,125.00",2021-09-07,2021-09-23
PTT,0.8200,"9,000","7,380.00","6,642.00",2021-09-29,2021-10-15


In [4]:
sql = '''
SELECT name, volbuy AS shares, price AS unit_cost, volbuy * price AS cost_amt 
FROM buy'''
df_buy = pd.read_sql(sql, const)
df_buy['shares'] = df_buy.shares.astype(int)
df_buy.dtypes

name          object
shares         int32
unit_cost    float64
cost_amt     float64
dtype: object

In [5]:
colu = 'name qtr shares div_amt net_div xd_date pay_date unit_cost pct'.split()

In [6]:
df_merge = pd.merge(df_div, df_buy, on='name', how='inner')
df_merge['pct'] = round(df_merge['net_div'] / df_merge['cost_amt'] * 100,2)
df_merge.rename(columns={'shares_x': 'shares'}, inplace=True)
df_merge[colu].sort_values(by=['pct','name'],ascending=[False,True]).style.format(format_dict)

,name,qtr,shares,div_amt,net_div,xd_date,pay_date,unit_cost,pct
13,NOBLE,0.5000,"63,000","31,500.00","28,350.00",2021-08-24,2021-09-07,7.00,6.43%
23,TISCO,6.3000,"2,000","12,600.00","11,340.00",2021-04-28,2021-05-17,97.50,5.82%
22,STA,1.7500,"20,000","35,000.00","31,500.00",2021-11-19,2021-12-08,34.75,4.53%
24,TMT,0.5000,"27,000","13,500.00","12,150.00",2021-08-27,2021-09-10,10.50,4.29%
11,MCS,0.6000,"60,000","36,000.00","32,400.00",2021-11-25,2021-12-09,16.70,3.23%
16,RATCH,1.2500,"9,000","11,250.00","10,125.00",2021-09-07,2021-09-23,45.75,2.46%
21,SIS,1.0000,"1,500","1,500.00","1,350.00",2021-03-10,2021-05-14,37.50,2.40%
8,JASIF,0.2500,"100,000","25,000.00","22,500.00",2021-11-19,2021-12-08,10.00,2.25%
27,VNT,0.9000,"4,500","4,050.00","3,645.00",2021-05-21,2021-05-21,37.50,2.16%
19,SAT,0.4800,"7,500","3,600.00","3,240.00",2021-08-25,2021-09-07,21.10,2.05%


In [7]:
round(df_merge.net_div.sum(),0)

226117.0

In [8]:
file_name = 'div-21q4.xlsx'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
df_merge.reset_index
df_merge[colu].sort_values(by=['xd_date','name'],ascending=[True,True]).to_excel(output_file, index=False)
df_merge[colu].sort_values(by=['xd_date','name'],ascending=[True,True]).to_excel(data_file, index=False)
df_merge[colu].sort_values(by=['xd_date','name'],ascending=[True,True]).\
to_excel(box_file, index=False, sheet_name='21Q4', float_format="%.2f")

In [9]:
file_name = 'div-21q4.xlsx'
one_path = "\\Users\\User\\OneDrive\\A8\\Excel\\"
one_file = one_path + file_name
df_merge.reset_index
df_merge[colu].sort_values(by=['xd_date','name'],ascending=[True,True]).\
to_excel(one_file, index=False, sheet_name='21Q4', float_format="%.2f")

### Past 2 weeks price, max, min to determine selling, buying prices

In [35]:
name = 'TISCO'
name

'TISCO'

In [36]:
sql = """
SELECT *
FROM price
WHERE name = '%s'
ORDER BY date ASC"""
sql = sql % name
print(sql)


SELECT *
FROM price
WHERE name = 'TISCO'
ORDER BY date ASC


In [37]:
df_price = pd.read_sql(sql, const)
df_price.shape

(1374, 7)

In [38]:
df_tmp = df_price.tail(10)
df_tmp

,name,date,price,maxp,minp,qty,opnp
1364,TISCO,2021-12-15,91.75,92.50,91.50,3450640,91.75
1365,TISCO,2021-12-16,92.75,93.00,92.00,4829802,92.25
1366,TISCO,2021-12-17,93.00,93.00,92.25,2916651,92.75
1367,TISCO,2021-12-20,92.00,93.00,91.75,3956929,92.50
1368,TISCO,2021-12-21,93.00,93.50,92.25,3863063,92.25
1369,TISCO,2021-12-22,93.50,94.00,93.25,3628161,93.25
1370,TISCO,2021-12-23,94.50,94.50,93.75,4747292,94.00
1371,TISCO,2021-12-24,94.25,94.75,94.00,1421626,94.50
1372,TISCO,2021-12-27,94.25,94.75,94.00,2979189,94.50
1373,TISCO,2021-12-28,95.25,95.25,94.50,4319367,94.50


In [39]:
df_tmp.maxp.value_counts(normalize=True)

93.00    0.3
94.75    0.2
92.50    0.1
93.50    0.1
94.00    0.1
94.50    0.1
95.25    0.1
Name: maxp, dtype: float64

In [40]:
df_tmp.minp.value_counts(normalize=True)

92.25    0.2
94.00    0.2
91.50    0.1
92.00    0.1
91.75    0.1
93.25    0.1
93.75    0.1
94.50    0.1
Name: minp, dtype: float64

In [41]:
df_tmp.maxp.max(),df_tmp.minp.min()

(95.25, 91.5)

### Change purchase date

In [4]:
sql = "UPDATE buy SET date = '2021-05-10' WHERE name = 'LH' AND active=1"
rp = const.execute(sql)
rp.rowcount

1

### List stocks with 5% yield to input x-date

In [14]:
sql = """
SELECT D.NAME AS name, Q4 AS qtr, SHARES AS shares, Q4*SHARES AS div_amt,
(Q4*SHARES*.9) AS net_div, xdate as xd_date, paiddate AS pay_date, P.price,
ROUND(DIVIDEND/P.PRICE*100,2) AS yield
FROM dividend D
JOIN price P
ON D.name = P.name
WHERE Q4 > 0 AND date = '%s'
ORDER BY D.name"""
sql = sql % today
print(sql)


SELECT D.NAME AS name, Q4 AS qtr, SHARES AS shares, Q4*SHARES AS div_amt,
(Q4*SHARES*.9) AS net_div, xdate as xd_date, paiddate AS pay_date, P.price,
ROUND(DIVIDEND/P.PRICE*100,2) AS yield
FROM dividend D
JOIN price P
ON D.name = P.name
WHERE Q4 > 0 AND date = '2021-12-29'
ORDER BY D.name


In [25]:
df_tmp = pd.read_sql(sql, const)
df_tmp.set_index('name', inplace=True)
df_tmp['shares'] = df_div['shares'].astype(int)
df_tmp.sort_values(['yield'],ascending=[False]).style.format(format_dict)

,qtr,shares,div_amt,net_div,xd_date,pay_date,price,yield
name,,,,,,,,
STA,1.7500,"20,000","35,000.00","31,500.00",2021-11-19,2021-12-08,31.25,16.80%
STGT,2.0000,0,0.00,0.00,2021-11-19,2021-12-07,30.00,15.00%
NOBLE,0.5000,"63,000","31,500.00","28,350.00",2021-08-24,2021-09-07,6.15,13.82%
TMT,0.5000,"27,000","13,500.00","12,150.00",2021-08-27,2021-09-10,10.20,10.78%
MCS,0.6000,"60,000","36,000.00","32,400.00",2021-11-25,2021-12-09,14.80,9.26%
JASIF,0.2500,"100,000","25,000.00","22,500.00",2021-11-19,2021-12-08,10.30,9.22%
PDG,0.2200,0,0.00,0.00,2021-03-15,2021-05-20,3.96,8.08%
CPNCG,0.2515,0,0.00,0.00,2021-11-15,2021-12-03,12.30,8.03%
WIIK,0.0900,0,0.00,0.00,2021-04-30,2021-05-21,2.38,7.98%


In [27]:
file_name = 'q4-xdate.xlsx'
one_path = "\\Users\\User\\OneDrive\\A8\\Excel\\"
one_file = one_path + file_name
#df_tmp.reset_index
df_tmp.sort_values(by=['yield','name'],ascending=[False,True]).\
to_excel(one_file, sheet_name='21Q4', float_format="%.2f")